In [2]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from sklearn import utils
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import utils as u


In [3]:
#Read the dataset:
df = pd.read_csv("NSE-TATA.csv")
df.head()

#Analyze the closing prices from dataframe:+
df["Date"] = pd.to_datetime(df.Date,format="%Y-%m-%d")
df.index = df['Date']

#Sort the dataset on date time and # filter “Date” and “Close” columns:
data = df.sort_index(ascending=True,axis=0)
new_dataset = pd.DataFrame(index=range(0,len(df)),columns=['Date','Close'])
for i in range(0,len(data)):
    new_dataset["Date"][i] = data['Date'][i]
    new_dataset["Close"][i] = data["Close"][i]
# Normalize the new filtered dataset:
new_dataset.index = new_dataset.Date
new_dataset.drop("Date",axis=1,inplace=True)
dataset = new_dataset.values

#convert between 0 - 1
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(dataset)



# n previous close price wanna traning
number_of_previous_close_prices = 60

train_data,test_data = u.create_data(dataset,0.8)



x_train, y_train = u.create_x_y_train(train_data,number_of_previous_close_prices)
x_test, y_test = u.create_x_y_test(test_data,number_of_previous_close_prices)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
pca = PCA() 
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

247


In [4]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(928, 60) (928,)
(187, 60) (187, 1)


In [64]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [45]:
type(x_train[0][0])

numpy.float64

In [50]:
#convert y_train to multi class
lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)
y_test = lab_enc.fit_transform(y_test)


In [6]:
test = lab_enc.inverse_transform(y_train)
scaler.inverse_transform(test.reshape(-1,1))

In [6]:
#LogisticRegression
# parameters1 = [    
#         {
#             'penalty' : ['l1','l2','elasticnet'],
#             'C':np.logspace(-4, 4, 10),
#             'solver': ['saga','lbfgs','sag','newton-cg'],
#             'max_iter' : [100, 1000,2500, 5000]
#         }
#     ]
parameters1 = [    
        {
            'penalty' : ['l2'],
            'C':np.logspace(-4, 4, 10),
            'solver': ['lbfgs'],
            'max_iter' : [100, 1000,2500, 5000]
        }
    ]
#RandomForestClassifier
parameters2 = [    
        {
            'n_estimators': [200, 300, 500,750, 800, 1200], #The number of trees in the forest.
            'max_features': ['auto', 'sqrt', 'log2'] #The number of features to consider when looking for the best split:
        }
    ]
#SVC
parameters3 = [    
        {
            'C':np.logspace(-4, 4, 10), #Regularization parameter. The strength of the regularization is inversely proportional to C.
            'gamma': ['scale', 'auto'], #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
            'kernel': ['rbf', 'poly', 'sigmoid'] #specifies the kernel type to be used in the algorithm
        }
    ]
#KNeighborsClassifier
parameters4 = [    
        {
            "n_neighbors":np.arange(20)+1, #Number of neighbors 
            "leaf_size":np.arange(25,40)+1
        }
    ]
#DecisionTreeClassifier
parameters5 = [    
        {
            "criterion":['gini', 'entropy'],
            "max_depth":np.linspace(1, 32, 32, endpoint=True)  #The maximum depth of the tree
        }
    ]

parameters = {}
parameters['LogisticRegression'] = parameters1
parameters['RandomForestClassifier'] = parameters2
parameters['SVC'] = parameters3
parameters['KNeighborsClassifier'] = parameters4
parameters['DecisionTreeClassifier'] = parameters5

In [58]:
parameters

{'LogisticRegression': [{'penalty': ['l2'],
   'C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
          3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
          1.29154967e+03, 1.00000000e+04]),
   'solver': ['lbfgs'],
   'max_iter': [100, 1000, 2500, 5000]}],
 'RandomForestClassifier': [{'n_estimators': [200, 300, 500, 750, 800, 1200],
   'max_features': ['auto', 'sqrt', 'log2']}],
 'SVC': [{'C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
          3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
          1.29154967e+03, 1.00000000e+04]),
   'gamma': ['scale', 'auto'],
   'kernel': ['rbf', 'poly', 'sigmoid']}],
 'KNeighborsClassifier': [{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20]),
   'leaf_size': array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])}],
 'DecisionTreeClassifier': [{'criterion': ['gini', 'entropy'],

In [10]:
y_train = np.float32(y_train)
type(y_train[1])

numpy.float32

In [12]:
y_train = y_train.reshape((-1, 1))
model = LinearRegression()
model.fit(x_train, y_train)
r_sq = model.score(x_train, y_train)
print('coefficient of determination:', r_sq)


coefficient of determination: 0.9799280228917286


In [19]:
y_pred = model.predict(x_test)
rmse = np.sqrt(np.mean(y_pred - y_test)**2)
print('rmse: ', rmse)


rmse:  0.22917667553843069


In [65]:
logisticRegr = LogisticRegression(n_jobs = -1)
logisticRegr.fit(x_train, y_train)
predictions = logisticRegr.predict(x_test) #test_img_pca #test_pca

In [66]:
predictions

array([209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,
       209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,
       209, 209, 209, 209, 161, 161, 161, 161, 161, 161, 161, 161, 161,
       161, 161, 161, 161, 161, 161, 161, 161, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 164, 164, 161, 161, 161, 161, 164, 164,
       164, 164, 164, 164, 209, 161, 161, 161, 161, 161, 161, 161, 164,
       164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 161, 161, 161, 161, 161, 161, 161, 161, 161, 161,
       161, 161, 161, 161, 161, 164, 164, 164, 164, 164, 164, 164, 164,
       164, 164, 164, 164, 164, 161, 161, 164, 164, 164, 161, 16

In [28]:

for element in parameters.keys():
    model = element
    if(model == 'LogisticRegression'):
        grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid = parameters[element] , scoring = 'accuracy', cv = 'None', n_jobs = -1 , verbose = 2)
        grid_scores = grid_search.fit(x_train , y_train)
    if(model == 'RandomForestClassifier'):
        grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = parameters[element] , scoring = 'accuracy', cv = 'None', n_jobs = -1 , verbose = 2)
        grid_scores = grid_search.fit(x_train , y_train)
    if(model == 'SVC'):
        grid_search = GridSearchCV(estimator = SVC(), param_grid = parameters[element] , scoring = 'accuracy', cv = 'None', n_jobs = -1 , verbose = 2)
        grid_scores = grid_search.fit(x_train , y_train)
    if(model == 'KNeighborsClassifier'):
        grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = parameters[element] , scoring = 'accuracy', cv = 'None', n_jobs = -1 , verbose = 2)
        grid_scores = grid_search.fit(x_train , y_train)
    if(model == 'DecisionTreeClassifier'):
        grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = parameters[element] , scoring = 'accuracy', cv = 'None', n_jobs = -1 , verbose = 2)
        grid_scores = grid_search.fit(x_train , y_train)
    
    print( grid_search)
    print( grid_search.best_score_)
    print( grid_search.best_params_)
    u.save_model(grid_search,f'{element}.txt')

Fitting 3 folds for each of 40 candidates, totalling 120 fits


C:\Python\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]),
                          'max_iter': [100, 1000, 2500, 5000],
                          'penalty': ['l2'], 'solver': ['lbfgs']}],
             scoring='accuracy', verbose=2)
0.015092041618818946
{'C': 10000.0, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
Fitting 3 folds for each of 18 candidates, totalling 54 fits
GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'max_features': ['auto', 'sqrt', 'log2'],
                          'n_estimators': [200, 300, 500, 750, 800, 1200]}],
             scoring='accuracy', verbose=2)
0.01185927549848627
{'max_features': 'log2', 'n_estimators': 200}
Fitting 3 folds for each of 60 candidates, totalling 180 fits


C:\Python\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]),
                          'gamma': ['scale', 'auto'],
                          'kernel': ['rbf', 'poly', 'sigmoid']}],
             scoring='accuracy', verbose=2)
0.016167310436023246
{'C': 2.782559402207126, 'gamma': 'scale', 'kernel': 'rbf'}
Fitting 3 folds for each of 300 candidates, totalling 900 fits


C:\Python\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid=[{'leaf_size': array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]),
                          'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])}],
             scoring='accuracy', verbose=2)
0.009698298360998017
{'leaf_size': 26, 'n_neighbors': 1}
Fitting 3 folds for each of 64 candidates, totalling 192 fits


C:\Python\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32.])}],
             scoring='accuracy', verbose=2)
0.01616383060166336
{'criterion': 'entropy', 'max_depth': 15.0}


In [43]:
from sklearn import metrics
# params_log = u.load_model('LogisticRegression.txt').best_params_
# params_log
svcc = SVC()
svcc.fit(x_train,y_train)
y_pred = svcc.predict(x_test)
print("SVC Accuracy:",metrics.accuracy_score(y_test, y_pred))

SVC Accuracy: 0.0053475935828877


In [44]:
y_test

array([168, 169, 175, 173, 172, 170, 177, 176, 174, 171, 164, 166, 167,
       165, 163, 163, 161, 162, 154, 151, 119, 112,  92, 136, 137, 131,
       144, 140, 133, 124,  98,  78,  91,  76, 106, 114, 120, 123, 119,
       125, 116, 100, 113,  99, 126, 130, 132, 134, 109,  71,  74,  79,
        69,  66,  77,  83,  68, 127, 118, 105, 111, 110, 129, 123, 115,
       115, 123, 135, 138, 143, 146, 147, 142, 141, 152, 155, 157, 160,
       158, 148, 150, 159, 153, 156, 145, 149,  63,  48,  45,  58,  53,
        52,  57,  60,  61,  65,  72,  64,  86, 101,  80,  62,  82,  89,
        95,  93,  81,  97, 103, 129, 108, 122,  94,  73,  67,  75,  86,
        96,  88,  84, 102,  85, 139, 128, 107,  90, 121, 117, 104,  87,
        70,  50,  59,  40,  13,  39,  51,  55,  34,  41,  47,  46,  56,
        28,  27,  24,  43,  38,  31,  36,  17,  22,  37,  42,  49,  54,
        44,  27,  12,  35,  16,  22,  29,  21,  33,   9,   6,   4,   8,
         5,   2,   7,  19,  32,  26,  25,  23,  15,  30,  20,  1

In [45]:
y_pred

array([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29],
      dtype=int64)

In [47]:
y_pred_t = lab_enc.inverse_transform(y_pred)
scaler.inverse_transform(y_pred_t.reshape(-1,1))

array([[6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [6572.55],
       [65

In [46]:
x_test

array([[0.4742268 , 0.47848498, 0.49013895, ..., 0.94240251, 0.93455849,
        0.93478261],
       [0.47848498, 0.49013895, 0.48229494, ..., 0.93455849, 0.93478261,
        0.93948902],
       [0.49013895, 0.48229494, 0.47938144, ..., 0.93478261, 0.93948902,
        0.94262662],
       ...,
       [0.75571493, 0.73352757, 0.77364411, ..., 0.58762887, 0.57485433,
        0.56006275],
       [0.73352757, 0.77364411, 0.77140296, ..., 0.57485433, 0.56006275,
        0.51792918],
       [0.77364411, 0.77140296, 0.7492156 , ..., 0.56006275, 0.51792918,
        0.47758853]])